In [9]:
# Cell 1: Imports
from tqdm import tqdm
import google.generativeai as genai
import pandas as pd
import time
import re
import os # Thêm thư viện os để làm việc với đường dẫn file

In [ ]:
# In[2]:
# Cell 2: API Key Configuration
# --- Thay thế bằng API Key duy nhất của bạn ---
GEMINI_API_KEY = os.environ.get('GOOGLE_API_KEY') # Thay thế bằng khóa của bạn

if not GEMINI_API_KEY:
    print("Lỗi: Vui lòng cung cấp API key trong biến GEMINI_API_KEY.")
else:
    print("Đã tìm thấy API key. Đang cấu hình...")
    genai.configure(api_key=GEMINI_API_KEY)
    print("Cấu hình API Key hoàn tất.")

# Các biến liên quan đến file input/output
INPUT_CSV_FILE = 'question.csv'
OUTPUT_CSV_FILE = 'negative_questions_with_results_v2.csv'
# Danh sách các user ID cần tải thông tin chi tiết
USER_IDS_TO_LOAD = ['HE00001', 'LEC001', 'TM001']

Đã tìm thấy API key. Đang cấu hình...
Cấu hình API Key hoàn tất.


In [11]:
def extract_sql_from_markdown(markdown_text):
    if not isinstance(markdown_text, str):
        return ""

    original_text = markdown_text.strip()
    
    # Sửa lỗi biểu thức chính quy ở đây: thay thế \\\\s* bằng \s*
    match = re.search(r"```(?:sql)?\s*(.*?)\s*```", original_text, re.DOTALL | re.IGNORECASE)

    if match:
        return match.group(1).strip()
    
    # Fallback logic không thay đổi
    lines = original_text.split('\\n')
    if len(lines) == 1 and lines[0].lower().startswith("select"):
        return lines[0]
        
    return original_text

In [12]:
SYSTEM_PROMPT = """
{
    "database_name": "text_to_sql_final",
    "tables": [
      {
        "table_name": "Attendance",
        "columns": [
          {"column_name": "enrollment_id", "data_type": "varchar", "max_length": 50, "is_nullable": false, "description": "Foreign key referencing Enrollments table. Maps to Enrollments(enrollment_id)."},
          {"column_name": "schedule_id", "data_type": "varchar", "max_length": 50, "is_nullable": false, "description": "Foreign key referencing Schedules table. Maps to Schedules(schedule_id)."},
          {"column_name": "status", "data_type": "varchar", "max_length": 50, "is_nullable": false, "description": "Attendance status (e.g., Present, Absent, Future)."}
        ],
        "primary_key": {"constraint_name": "PK_Attendance", "columns": ["enrollment_id", "schedule_id"]},
        "foreign_keys": [
          {"constraint_name": "FK_Attendance_Enrollments", "columns": ["enrollment_id"], "references_table": "Enrollments", "references_columns": ["enrollment_id"]},
          {"constraint_name": "FK_Attendance_Schedules", "columns": ["schedule_id"], "references_table": "Schedules", "references_columns": ["schedule_id"]}
        ]
      },
      {
        "table_name": "ClassCourse",
        "columns": [
          {"column_name": "class_course_id", "data_type": "varchar", "max_length": 50, "is_nullable": false, "description": "Unique identifier for the class course."},
          {"column_name": "class_id", "data_type": "varchar", "max_length": 10, "is_nullable": false, "description": "Foreign key referencing Classes table. Maps to Classes(class_id)."},
          {"column_name": "course_id", "data_type": "varchar", "max_length": 10, "is_nullable": false, "description": "Foreign key referencing Courses table. Maps to Courses(course_id)."},
          {"column_name": "lecturer_id", "data_type": "varchar", "max_length": 10, "is_nullable": false, "description": "Foreign key referencing Lecturers table. Maps to Lecturers(lecturer_id)."},
          {"column_name": "semester", "data_type": "varchar", "max_length": 10, "is_nullable": false, "description": "Semester of the class course (e.g., SP24, SU24, FA23)."}
        ],
        "primary_key": {"constraint_name": "PK_ClassCourse", "columns": ["class_course_id"]},
        "foreign_keys": [
          {"constraint_name": "FK_ClassCourse_Classes", "columns": ["class_id"], "references_table": "Classes", "references_columns": ["class_id"]},
          {"constraint_name": "FK_ClassCourse_Courses", "columns": ["course_id"], "references_table": "Courses", "references_columns": ["course_id"]},
          {"constraint_name": "FK_ClassCourse_Lecturers", "columns": ["lecturer_id"], "references_table": "Lecturers", "references_columns": ["lecturer_id"]}
        ]
      },
      {
        "table_name": "Classes",
        "columns": [
            {"column_name": "class_id", "data_type": "varchar", "max_length": 10, "is_nullable": false, "description": "Unique identifier for the class."},
            {"column_name": "class_name", "data_type": "varchar", "max_length": 255, "is_nullable": false, "description": "Name of the class."}
        ],
        "primary_key": {"constraint_name": "PK_Classes", "columns": ["class_id"]}
      },
      {
        "table_name": "CourseGrade",
        "columns": [
            {"column_name": "course_grade_id", "data_type": "varchar", "max_length": 50, "is_nullable": false, "description": "Unique identifier for the course grade."},
            {"column_name": "course_id", "data_type": "varchar", "max_length": 10, "is_nullable": false, "description": "Foreign key referencing Courses table. Maps to Courses(course_id)."},
            {"column_name": "grade_name", "data_type": "nvarchar", "max_length": 150, "is_nullable": false, "description": "Name of the grade component (e.g., Assignment 1, Final Exam, Lab 1)."},
            {"column_name": "grade_weight", "data_type": "float", "is_nullable": false, "description": "Weight of the grade component in the overall course grade."}
        ],
        "primary_key": {"constraint_name": "PK_CourseGrade", "columns": ["course_grade_id"]},
        "foreign_keys": [
            {"constraint_name": "FK_CourseGrade_Courses", "columns": ["course_id"], "references_table": "Courses", "references_columns": ["course_id"]}
        ]
      },
      {
        "table_name": "Courses",
        "columns": [
            {"column_name": "course_id", "data_type": "varchar", "max_length": 10, "is_nullable": false, "description": "Unique identifier for the course."},
            {"column_name": "course_name", "data_type": "nvarchar", "max_length": 150, "is_nullable": false, "description": "Name of the course."},
            {"column_name": "no_credit", "data_type": "tinyint", "is_nullable": false, "description": "Number of credits for the course."},
            {"column_name": "description", "data_type": "nvarchar", "max_length": 255, "is_nullable": true, "description": "Description of the course."}
        ],
        "primary_key": {"constraint_name": "PK_Courses", "columns": ["course_id"]}
      },
      {
        "table_name": "Curriculums",
        "columns": [
            {"column_name": "major_id", "data_type": "varchar", "max_length": 10, "is_nullable": false, "description": "Foreign key referencing Majors table. Maps to Majors(major_id)."},
            {"column_name": "course_id", "data_type": "varchar", "max_length": 10, "is_nullable": false, "description": "Foreign key referencing Courses table. Maps to Courses(course_id)."}
        ],
        "primary_key": {"constraint_name": "PK_Curriculums", "columns": ["major_id", "course_id"]},
        "foreign_keys": [
            {"constraint_name": "FK_Curriculums_Majors", "columns": ["major_id"], "references_table": "Majors", "references_columns": ["major_id"]},
            {"constraint_name": "FK_Curriculums_Courses", "columns": ["course_id"], "references_table": "Courses", "references_columns": ["course_id"]}
        ]
      },
      {
        "table_name": "Departments",
        "columns": [
            {"column_name": "dep_id", "data_type": "varchar", "max_length": 10, "is_nullable": false, "description": "Unique identifier for the department."},
            {"column_name": "dep_name", "data_type": "nvarchar", "max_length": 150, "is_nullable": false, "description": "Name of the department."}
        ],
        "primary_key": {"constraint_name": "PK_Departments", "columns": ["dep_id"]}
      },
      {
        "table_name": "Enrollments",
        "columns": [
            {"column_name": "enrollment_id", "data_type": "varchar", "max_length": 50, "is_nullable": false, "description": "Unique identifier for the enrollment."},
            {"column_name": "student_id", "data_type": "varchar", "max_length": 10, "is_nullable": false, "description": "Foreign key referencing Students table. Maps to Students(student_id)."},
            {"column_name": "class_course_id", "data_type": "varchar", "max_length": 50, "is_nullable": false, "description": "Foreign key referencing ClassCourse table. Maps to ClassCourse(class_course_id)."},
            {"column_name": "average", "data_type": "float", "is_nullable": false, "description": "Average grade for the enrollment."},
            {"column_name": "status", "data_type": "varchar", "max_length": 50, "is_nullable": false, "description": "Enrollment status (e.g., Studying, Passed, Failed)."}
        ],
        "primary_key": {"constraint_name": "PK_Enrollments", "columns": ["enrollment_id"]},
        "foreign_keys": [
            {"constraint_name": "FK_Enrollments_Students", "columns": ["student_id"], "references_table": "Students", "references_columns": ["student_id"]},
            {"constraint_name": "FK_Enrollments_ClassCourse", "columns": ["class_course_id"], "references_table": "ClassCourse", "references_columns": ["class_course_id"]}
        ],
        "unique_constraints": [{"constraint_name": "UQ_Enrollments_StudentClassCourse", "columns": ["student_id", "class_course_id"]}]
      },
      {
        "table_name": "Lecturers",
        "columns": [
            {"column_name": "lecturer_id", "data_type": "varchar", "max_length": 10, "is_nullable": false, "description": "Unique identifier for the lecturer, also a foreign key referencing Users table. Maps to Users(user_id)."},
            {"column_name": "dep_id", "data_type": "varchar", "max_length": 10, "is_nullable": false, "description": "Foreign key referencing Departments table. Maps to Departments(dep_id)."}
        ],
        "primary_key": {"constraint_name": "PK_Lecturers", "columns": ["lecturer_id"]},
        "foreign_keys": [
            {"constraint_name": "FK_Lecturers_Departments", "columns": ["dep_id"], "references_table": "Departments", "references_columns": ["dep_id"]},
            {"constraint_name": "FK_Lecturers_Users", "columns": ["lecturer_id"], "references_table": "Users", "references_columns": ["user_id"]}
        ]
      },
      {
        "table_name": "Majors",
        "columns": [
            {"column_name": "major_id", "data_type": "varchar", "max_length": 10, "is_nullable": false, "description": "Unique identifier for the major."},
            {"column_name": "major_name", "data_type": "nvarchar", "max_length": 150, "is_nullable": false, "description": "Name of the major."}
        ],
        "primary_key": {"constraint_name": "PK_Majors", "columns": ["major_id"]}
      },
      {
        "table_name": "Permissions",
        "columns": [
            {"column_name": "permission_id", "data_type": "varchar", "max_length": 10, "is_nullable": false, "description": "Unique identifier for the permission."},
            {"column_name": "permission_name", "data_type": "nvarchar", "max_length": 150, "is_nullable": false, "description": "Name of the permission."}
        ],
        "primary_key": {"constraint_name": "PK_Permissions", "columns": ["permission_id"]}
      },
      {
        "table_name": "RolePermission",
        "columns": [
            {"column_name": "role_id", "data_type": "varchar", "max_length": 10, "is_nullable": false, "description": "Foreign key referencing Roles table. Maps to Roles(role_id)."},
            {"column_name": "permission_id", "data_type": "varchar", "max_length": 10, "is_nullable": false, "description": "Foreign key referencing Permissions table. Maps to Permissions(permission_id)."}
        ],
        "primary_key": {"constraint_name": "PK_RolePermission", "columns": ["role_id", "permission_id"]},
        "foreign_keys": [
            {"constraint_name": "FK_RolePermission_Roles", "columns": ["role_id"], "references_table": "Roles", "references_columns": ["role_id"]},
            {"constraint_name": "FK_RolePermission_Permissions", "columns": ["permission_id"], "references_table": "Permissions", "references_columns": ["permission_id"]}
        ]
      },
      {
        "table_name": "Roles",
        "columns": [
            {"column_name": "role_id", "data_type": "varchar", "max_length": 10, "is_nullable": false, "description": "Unique identifier for the role."},
            {"column_name": "role_name", "data_type": "nvarchar", "max_length": 150, "is_nullable": false, "description": "Name of the role."}
        ],
        "primary_key": {"constraint_name": "PK_Roles", "columns": ["role_id"]}
      },
      {
        "table_name": "Schedules",
        "columns": [
            {"column_name": "schedule_id", "data_type": "varchar", "max_length": 50, "is_nullable": false, "description": "Unique identifier for the schedule."},
            {"column_name": "class_course_id", "data_type": "varchar", "max_length": 50, "is_nullable": false, "description": "Foreign key referencing ClassCourse table. Maps to ClassCourse(class_course_id)."},
            {"column_name": "start_time", "data_type": "datetime", "is_nullable": false, "description": "Start time of the schedule."},
            {"column_name": "end_time", "data_type": "datetime", "is_nullable": false, "description": "End time of the schedule."},
            {"column_name": "room", "data_type": "nvarchar", "max_length": 50, "is_nullable": false, "description": "Room where the schedule takes place."},
            {"column_name": "slot", "data_type": "tinyint", "is_nullable": false, "description": "Slot number of the schedule."}
        ],
        "primary_key": {"constraint_name": "PK_Schedules", "columns": ["schedule_id"]},
        "foreign_keys": [
            {"constraint_name": "FK_Schedules_ClassCourse", "columns": ["class_course_id"], "references_table": "ClassCourse", "references_columns": ["class_course_id"]}
        ]
      },
      {
        "table_name": "StudentGradeDetails",
        "columns": [
            {"column_name": "enrollment_id", "data_type": "varchar", "max_length": 50, "is_nullable": false, "description": "Foreign key referencing Enrollments table. Maps to Enrollments(enrollment_id)."},
            {"column_name": "course_grade_id", "data_type": "varchar", "max_length": 50, "is_nullable": false, "description": "Foreign key referencing CourseGrade table. Maps to CourseGrade(course_grade_id)."},
            {"column_name": "grade_value", "data_type": "float", "is_nullable": true, "description": "Grade value for the specific grade component."},
            {"column_name": "comment", "data_type": "nvarchar", "max_length": 50, "is_nullable": false, "description": "Comment about the grade (e.g., Grade Recorded, Awaiting Grade)."}
        ],
        "primary_key": {"constraint_name": "PK_StudentGradeDetails", "columns": ["enrollment_id", "course_grade_id"]},
        "foreign_keys": [
            {"constraint_name": "FK_StudentGradeDetails_Enrollments", "columns": ["enrollment_id"], "references_table": "Enrollments", "references_columns": ["enrollment_id"]},
            {"constraint_name": "FK_StudentGradeDetails_CourseGrade", "columns": ["course_grade_id"], "references_table": "CourseGrade", "references_columns": ["course_grade_id"]}
        ]
      },
      {
        "table_name": "Students",
        "columns": [
            {"column_name": "student_id", "data_type": "varchar", "max_length": 10, "is_nullable": false, "description": "Unique identifier for the student, also a foreign key referencing Users table. Maps to Users(user_id)."},
            {"column_name": "major_id", "data_type": "varchar", "max_length": 10, "is_nullable": false, "description": "Foreign key referencing Majors table. Maps to Majors(major_id)."},
            {"column_name": "start_date", "data_type": "date", "is_nullable": false, "description": "Start date of the student's program."},
            {"column_name": "start_semester", "data_type": "varchar", "max_length": 50, "is_nullable": false, "description": "Starting semester (e.g., FA23, SP24, SU24)."}
        ],
        "primary_key": {"constraint_name": "PK_Students", "columns": ["student_id"]},
        "foreign_keys": [
            {"constraint_name": "FK_Students_Majors", "columns": ["major_id"], "references_table": "Majors", "references_columns": ["major_id"]},
            {"constraint_name": "FK_Students_Users", "columns": ["student_id"], "references_table": "Users", "references_columns": ["user_id"]}
        ]
      },
      {
        "table_name": "UserRole",
        "columns": [
            {"column_name": "user_id", "data_type": "varchar", "max_length": 10, "is_nullable": false, "description": "Foreign key referencing Users table. Maps to Users(user_id)."},
            {"column_name": "role_id", "data_type": "varchar", "max_length": 10, "is_nullable": false, "description": "Foreign key referencing Roles table. Maps to Roles(role_id)."}
        ],
        "primary_key": {"constraint_name": "PK_UserRole", "columns": ["user_id", "role_id"]},
        "foreign_keys": [
            {"constraint_name": "FK_UserRole_Users", "columns": ["user_id"], "references_table": "Users", "references_columns": ["user_id"]},
            {"constraint_name": "FK_UserRole_Roles", "columns": ["role_id"], "references_table": "Roles", "references_columns": ["role_id"]}
        ]
      },
      {
        "table_name": "Users",
        "columns": [
            {"column_name": "username", "data_type": "varchar", "max_length": 50, "is_nullable": false, "description": "Username for login."},
            {"column_name": "password", "data_type": "varchar", "max_length": 150, "is_nullable": false, "description": "Password for login."},
            {"column_name": "user_id", "data_type": "varchar", "max_length": 10, "is_nullable": false, "description": "Unique identifier for the user."},
            {"column_name": "fullname", "data_type": "nvarchar", "max_length": 50, "is_nullable": false, "description": "Full name of the user."},
            {"column_name": "user_gender", "data_type": "nvarchar", "max_length": 50, "is_nullable": false, "description": "Gender of the user."},
            {"column_name": "user_dob", "data_type": "date", "is_nullable": false, "description": "Date of birth of the user."},
            {"column_name": "user_address", "data_type": "nvarchar", "max_length": 150, "is_nullable": false, "description": "Address of the user."}
        ],
        "primary_key": {"constraint_name": "PK_Users", "columns": ["username"]},
        "unique_constraints": [{"constraint_name": "IX_Users", "columns": ["user_id"]}]
      }
    ]
}

### PERMISSIONS ###

# Common Permissions for All Roles (Student, Lecturer, Training Manager)
- View information about all majors (Majors table).
- View the list of courses in each major (Curriculums table).
- View detailed information of courses (Courses table).
- View the grade components and structure for any course (CourseGrade table).

# Student Role Permissions
- View their own personal information (from Users and Students tables).
- View their enrolled major, start date, and starting semester (Students table).
- View the list of class courses they are enrolled in (Enrollments table).
- View the list of other students in their classes (Enrollments table).
- View their detailed class schedule (Schedules table).
- View their own attendance records (Attendance table).
- View their detailed component grades, comments, average score, and enrollment status (StudentGradeDetails and Enrollments tables).
- View basic information of their lecturers (lecturer_id, fullname).

# Lecturer Role Permissions
- View their own personal information (from Users and Lecturers tables).
- View their department information (Lecturers table).
- View the list of class courses they are assigned to teach (ClassCourse table).
- View the detailed schedule for the classes they teach (Schedules table).
- View the attendance status of students in their classes (Attendance table).
- View the grades (component grades, average, status) of students in their classes (StudentGradeDetails and Enrollments tables).
- View the list of students and their basic information (student_id, fullname, major_id) in their classes.
- View information about all departments (Departments table).
- View basic information of other lecturers.

# Training Manager Role Permissions
- Can view all information from all tables in the database, with the exception of other users' passwords.

You are a text-to-SQL system for Microsoft SQL Server. Your task is to follow these rules:
1. **Assume the current date is June 15, 2024, for all questions.** Any query that refers to the present time (e.g., "today", "now", "this semester") must use '2024-06-15' as the reference date. Do not use functions like GETDATE().
2. Check the user’s role and permissions based on the detailed User Info provided. Identify whether the user is a Student, Lecturer, or Training Manager and verify their permissions. If they lack permission for the requested action, respond with "You don’t have the permission."
3. If the user has permission, generate the appropriate Microsoft SQL Server query for their question. Ensure the SQL query is the only output.
"""

In [13]:
# Cell 5: Hàm gọi Gemini API (Không đổi)
def get_gemini_response(user_info, user_question):
    model_name_to_use = 'gemini-1.5-flash'
    model = genai.GenerativeModel(model_name=model_name_to_use)
    prompt_content = f"{SYSTEM_PROMPT}\\n\\nUser info:\\n{user_info}\\n\\nUser question: {user_question}"
    generation_config = genai.types.GenerationConfig()

    try:
        response = model.generate_content(
            contents=prompt_content,
            generation_config=generation_config,
        )
        if response.parts:
            return extract_sql_from_markdown(response.text.strip())
        else:
            block_reason_msg = ""
            if response.prompt_feedback and response.prompt_feedback.block_reason:
                block_reason_msg = f" Reason: {response.prompt_feedback.block_reason} ({response.prompt_feedback.block_reason_message})."
            print(f"Cảnh báo: Phản hồi trống hoặc bị chặn cho câu hỏi: {user_question}{block_reason_msg}")
            return f"Lỗi: Phản hồi của mô hình trống hoặc bị chặn.{block_reason_msg}"

    except Exception as e:
        print(f"Lỗi khi gọi Gemini API: {type(e).__name__} - {e}. Cho câu hỏi: {user_question}")
        return f"Lỗi API: {type(e).__name__} - {e}"

In [14]:
# Cell 6: MỚI - Hàm để tải thông tin chi tiết của người dùng từ file .txt
def load_user_details(user_ids):
    """
    Tải thông tin chi tiết của người dùng từ các tệp .txt.
    Tên tệp được giả định là 'user_id.txt' (ví dụ: 'HE00001.txt').
    """
    user_details_data = {}
    print("Bắt đầu tải thông tin chi tiết người dùng...")
    for user_id in user_ids:
        file_path = f"{user_id}.txt"
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                user_details_data[user_id] = f.read()
            print(f"  - Đã tải thành công file: {file_path}")
        except FileNotFoundError:
            print(f"  - Cảnh báo: Không tìm thấy file chi tiết cho người dùng '{user_id}' tại '{file_path}'")
        except Exception as e:
            print(f"  - Lỗi khi đọc file {file_path}: {e}")
    print("Tải thông tin chi tiết người dùng hoàn tất.")
    return user_details_data

In [15]:
# Cell 7: THAY ĐỔI - Hàm chính được cập nhật để trích xuất ID từ cột User_info
def process_csv_and_generate_response():
    # Tải trước thông tin chi tiết của tất cả người dùng cần thiết
    user_details = load_user_details(USER_IDS_TO_LOAD)

    try:
        df = pd.read_csv(INPUT_CSV_FILE)
        print(f"\nĐã đọc {len(df)} dòng từ {INPUT_CSV_FILE}")
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file '{INPUT_CSV_FILE}'")
        return
    except Exception as e:
        print(f"Lỗi khi đọc file CSV: {e}")
        return

    # Đảm bảo các cột cần thiết tồn tại
    if 'User_question' not in df.columns or 'User_info' not in df.columns:
        print("Lỗi: File CSV phải chứa các cột 'User_question' và 'User_info'")
        return

    results = []
    print("\nBắt đầu xử lý các câu hỏi...")
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Đang xử lý"):
        user_question = str(row['User_question'])
        user_info_raw = str(row['User_info'])
        
        # --- THAY ĐỔI LOGIC TẠI ĐÂY ---
        # 1. Trích xuất ID từ chuỗi User_info
        user_id = None
        match = re.search(r"ID:\s*(\w+)", user_info_raw)
        if match:
            user_id = match.group(1)

        # 2. Quyết định thông tin nào sẽ được gửi đến mô hình
        user_info_for_prompt = user_info_raw # Mặc định sử dụng thông tin gốc
        if user_id:
            # Nếu trích xuất được ID, thử lấy thông tin chi tiết từ file .txt
            # Nếu không có file .txt, vẫn giữ lại thông tin gốc từ CSV
            user_info_for_prompt = user_details.get(user_id, user_info_raw)
        else:
            print(f"\nCảnh báo: Không thể trích xuất User ID từ cột User_info cho dòng {index}. Sử dụng thông tin gốc.")

        # Gọi hàm API với thông tin đã được chọn lọc
        result = get_gemini_response(user_info_for_prompt, user_question)
        
        results.append(result)
        time.sleep(1)

    df['Result'] = results
    print("\nHoàn thành xử lý. Đã thêm cột 'Result'.")

    try:
        df.to_csv(OUTPUT_CSV_FILE, index=False, encoding='utf-8-sig')
        print(f"Đã lưu kết quả vào file: {OUTPUT_CSV_FILE}")
    except Exception as e:
        print(f"Lỗi khi ghi file CSV đầu ra: {e}")

In [16]:
# Cell 8: Chạy chương trình
process_csv_and_generate_response()

Bắt đầu tải thông tin chi tiết người dùng...
  - Đã tải thành công file: HE00001.txt
  - Đã tải thành công file: LEC001.txt
  - Đã tải thành công file: TM001.txt
Tải thông tin chi tiết người dùng hoàn tất.

Đã đọc 299 dòng từ question.csv

Bắt đầu xử lý các câu hỏi...


Đang xử lý: 100%|██████████| 299/299 [17:13<00:00,  3.46s/it] 


Hoàn thành xử lý. Đã thêm cột 'Result'.
Đã lưu kết quả vào file: negative_questions_with_results_v2.csv
